In [55]:
import pickle, fileinput
import json

In [ ]:
# CONFIG ZONE
# YOUR LOCAL SYSTEM INFO
dir_path = 'replays_wl/'
files = ["770", "775", "781", "782", "783", "784", "869", "875", "895", "913", "922", "925", "1478"]
map_path = 'winter_steps.png'
warm_map_template_name = 'ws'

# MAP INFO
hexes_width = 44
hexes_hight = 40

# NEED SOME TESTS IN NEXT CELLS
main_forward_shift = 27 # cool
main_down_shift = 36 # cool
main_shift =  (27, 36)
pixel_shift_odd = (10, 32)
pixel_shift_even = (10, 48)


generate_all_maps(dir_path=dir_path, files=files, map_path=map_path, new_map_name=warm_map_template_name,
                  show=False, hexes_width=hexes_width, hexes_hight=hexes_hight,
                  mark_size=15, pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift, alpha=0.65
)

In [ ]:
# CONFIG ZONE
# YOUR LOCAL SYSTEM INFO
dir_path = 'replays_wl/'
files = ["370"
,"371"
,"426"
,"466"
,"542"
,"564"
,"584"
,"626"
,"642"
,"669"
,"697"
,"706"
,"709"
,"735"
,"787"
,"794"
,"863"
,"905"
,"908"
,"916"
,"920"
,"928"
,"1012"
,"1061"
,"1075"
,"1089"
,"1091"
,"1107"
,"1112"
,"1194"
,"1230"
,"1231"
,"1240"
,"1241"
,"1275"
,"1312"
,"1327"
,"1336"
,"1475"
,"1481"]
map_path = 'cold_war.png'
warm_map_template_name = 'cw'

# MAP INFO
hexes_width = 50
hexes_hight = 46

# NEED SOME TESTS IN NEXT CELLS
main_forward_shift = 27 # cool
main_down_shift = 36 # cool
main_shift =  (54, 72)
pixel_shift_odd = (47, 46)
pixel_shift_even = (47, 85)



In [ ]:
# CONFIG ZONE
# YOUR LOCAL SYSTEM INFO
dir_path = 'replays_wl/'
files = ["343",
"354",
"356",
"358",
"363",
"410",
"416",
"423",
"560",
"573",
"599",
"612",
"629",
"650",
"651",
"657",
"685",
"698",
"788",
"789",
"792",
"860",
"861",
"866",
"867",
"872",
"884",
"888",
"889",
"899",
"900",
"906",
"924",
"929",
"1019",
"1025",
"1039",
"1082",
"1108",
"1266"]
map_path = 'autumn_harvest.png'
warm_map_template_name = 'ah'

# MAP INFO
hexes_width = 44
hexes_hight = 32

# NEED SOME TESTS IN NEXT CELLS
mark_size = 20
main_shift =  (54, 72)
pixel_shift_odd = (47, 50)
pixel_shift_even = (47, 92)

In [56]:
import sys, os


In [136]:
from PIL import Image, ImageDraw
from collections import defaultdict

def extract_commands(file):
    pre_parts = file.split('[command]')
    parts = [pre_part.split('[/command]')[0] for pre_part in pre_parts if '[/command]' in pre_part]
    return parts

def count_moves_heat(command, cache, mode='all'):
    if '[move]' not in command:
        return []
    if cache and 'inited_side' in cache:
        if int(cache['inited_side']) != int(cache['which_side']):
            return []
    x_co = command.split('x=\"')[1].split('\"\n')[0].split(',')
    y_co = command.split('y=\"')[1].split('\"\n')[0].split(',')
    if mode == 'all':
        return list(zip(x_co, y_co))[1:]
    if mode == 'finish':
        return [[x_co[0], y_co[0]], [x_co[-1], y_co[-1]]]

def count_recruit_heat(command, cache):
    if '[recruit]' not in command:
        return []
    if cache and 'inited_side' in cache:
        if int(cache['inited_side']) != int(cache['which_side']):
            return []
    x_co = command.split('x=')[1].split('\n')[0]
    y_co = command.split('y=')[1].split('\n')[0]
    return [[x_co, y_co]]

def count_fight_heat(command, cache):
    if '[attack]' not in command:
        return []
    if cache and 'inited_side' in cache:
        if int(cache['inited_side']) != int(cache['which_side']):
            return []
    x_at = command.split('x=')[1].split('\n')[0].split(',')
    y_at = command.split('y=')[1].split('\n')[0].split(',')
    x_def = command.split('x=')[2].split('\n')[0].split(',')
    y_def = command.split('y=')[2].split('\n')[0].split(',')
    return [[x_at[0], y_at[0]], [x_def[0], y_def[0]]]

def count_attack_heat(command, cache):
    heat = count_fight_heat(command, cache)
    if heat:
        return [heat[0]]
    return []

def count_defence_heat(command, cache):
    heat = count_fight_heat(command, cache)
    if heat:
        return [heat[1]]
    return []

def count_mv_rc_heat(command, cache):
    return count_moves_heat(command, cache) + count_recruit_heat(command, cache)

def count_color(hot, max_hot, mult = 1):
    hott = min(int(255 * mult * hot / max_hot), 255)
    if hott == 0:
        return (0, 0, 0)
    return (hott, 0, 255 - hott)

def count_color_2(hot, max_hot, mult = 1):
    hott = min(int(255 * mult * hot / max_hot), 255)
    return (hott, 0, 255 - hott) if hott else None

def to_hex(i, j):
    return str(i) + "_" + str(j)

def no_cache(command, cache):
    if not cache:
        return {}

def cache_side(command, cache):
    if not cache:
        return {}
    if '[init_side]' in command:
        try:
            cache['inited_side'] = command.split('side_number=')[1].split('\n')[0]
        except:
            pass
#             print(command)
    return cache


def generate_single_heat_map(dir_path, files, map_path, new_map_name, 
                        count_heat_by_command,
                        hexes_width, hexes_hight, multiplier,
                        mark_size, pixel_shift_odd, pixel_shift_even, main_shift,
                        alpha=0.65, show=False,  players_mod='both', count_cache_by_command=no_cache
):
    warm_map = defaultdict(int)
    cool_map_ids = set()
    for i in range(1, hexes_width):
        for j in range(1, hexes_hight):
            warm_map[str(i) + '_' + str(j)] = 0
    cache = {}
    if players_mod != 'both':
        cache['which_side'] = players_mod 
    for file in files:
        filename = dir_path + file
        with open(filename, 'r') as f:
            text = f.read()
        commands = extract_commands(text)
        for command in commands:
            cache = count_cache_by_command(command, cache)
            heat_hexes = count_heat_by_command(command, cache)
            if heat_hexes:
                cool_map_ids.add(file)
            for hexx in heat_hexes:
                warm_map[to_hex(hexx[0], hexx[1])] += 1
            
    image = Image.open(map_path)
    image_copy = image.copy()

    max_hot = max(warm_map.values())
    for i in range(1, hexes_width, 2):
        for j in range(1, hexes_hight):
            color = count_color_2(warm_map[to_hex(i, j)], max_hot, multiplier)
            if color:
                canvas = Image.new('RGB', (mark_size, mark_size), color)
                image_copy.paste(canvas, (pixel_shift_odd[0] + (i - 1) * main_shift[0],
                                          pixel_shift_odd[1] + (j - 1) * main_shift[1]))

    for i in range(2, hexes_width, 2):
        for j in range(1, hexes_hight):
            color = count_color_2(warm_map[to_hex(i, j)], max_hot, multiplier)
            if color:
                canvas = Image.new('RGB', (mark_size, mark_size), color)
                image_copy.paste(canvas, (pixel_shift_even[0] + (i - 1) * main_shift[0],
                                          pixel_shift_even[1] + (j - 1) * main_shift[1]))
    
    image = Image.blend(image, image_copy, alpha=alpha)
    if new_map_name:
        image.save(new_map_name)
    if show:
        image.show()
    print(cool_map_ids)

In [137]:
def generate_all_maps(dir_path, files, map_path, new_map_name,
                        hexes_width, hexes_hight, 
                        mark_size, pixel_shift_odd, pixel_shift_even, main_shift,
                      alpha=0.65, show=False
):
    generate_single_heat_map(dir_path=dir_path, files=files, map_path=map_path, new_map_name=new_map_name + '_mv_rc_2.png',
                             count_heat_by_command=count_mv_rc_heat,
                             hexes_width=hexes_width, hexes_hight=hexes_hight, multiplier=2,
                             mark_size=mark_size,
                             pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift,
                             alpha=alpha, show=show, players_mod='both')
    generate_single_heat_map(dir_path=dir_path, files=files, map_path=map_path, new_map_name=new_map_name + '_mv_rc_p1_3.png',
                             count_heat_by_command=count_mv_rc_heat,
                             hexes_width=hexes_width, hexes_hight=hexes_hight, multiplier=3,
                             mark_size=mark_size,
                             pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift,
                             alpha=alpha, show=show, players_mod='1', count_cache_by_command=cache_side)
    generate_single_heat_map(dir_path=dir_path, files=files, map_path=map_path, new_map_name=new_map_name + '_mv_rc_p2_3.png',
                             count_heat_by_command=count_mv_rc_heat,
                             hexes_width=hexes_width, hexes_hight=hexes_hight, multiplier=3,
                             mark_size=mark_size,
                             pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift,
                             alpha=alpha, show=show, players_mod='2', count_cache_by_command=cache_side)
    generate_single_heat_map(dir_path=dir_path, files=files, map_path=map_path, new_map_name=new_map_name + '_def_p2_3.png',
                             count_heat_by_command=count_defence_heat,
                             hexes_width=hexes_width, hexes_hight=hexes_hight, multiplier=3,
                             mark_size=mark_size,
                             pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift,
                             alpha=alpha, show=show, players_mod='1', count_cache_by_command=cache_side)
    generate_single_heat_map(dir_path=dir_path, files=files, map_path=map_path, new_map_name=new_map_name + '_def_p1_3.png',
                             count_heat_by_command=count_defence_heat,
                             hexes_width=hexes_width, hexes_hight=hexes_hight, multiplier=3,
                             mark_size=mark_size,
                             pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift,
                             alpha=alpha, show=show, players_mod='2', count_cache_by_command=cache_side)


In [59]:
def test_mark(mark_size, location, map_path):
    image = Image.open(map_path)
    image_copy = image.copy()
    canvas = Image.new('RGB', (mark_size, mark_size), 'cyan')
    image_copy.paste(canvas, location)
    image_copy.show()

In [73]:
def test_shift(shift, mark_size, location, map_path):
    image = Image.open(map_path)
    image_copy = image.copy()
    canvas = Image.new('RGB', (mark_size, mark_size), 'cyan')
    for i in range(0, 10, 2):
        for j in range(0, 10, 2):
            image_copy.paste(canvas, (location[0] + shift[0] * i, location[1] + shift[1] * j))
    image_copy.show()

In [61]:
def test_odd_hexes(location_odd, mark_size, shift, hexes_width, hexes_hight, map_path):
    image = Image.open(map_path)
    image_copy = image.copy()
    for i in range(1, hexes_width, 2):
        for j in range(1, hexes_hight):
            canvas = Image.new('RGB', (mark_size, mark_size), 'cyan')
            image_copy.paste(canvas, (location_odd[0] + (i - 1) * shift[0],
                                      location_odd[1] + (j - 1) * shift[1]))
    image_copy.show()

def test_even_hexes(location_even, mark_size, shift, hexes_width, hexes_hight, map_path):
    image = Image.open(map_path)
    image_copy = image.copy()
    for i in range(2, hexes_width, 2):
        for j in range(1, hexes_hight):
            canvas = Image.new('RGB', (mark_size, mark_size), 'cyan')
            image_copy.paste(canvas, (location_even[0] + (i - 1) * shift[0],
                                      location_even[1] + (j - 1) * shift[1]))
    image_copy.show()

def test_all(location_odd, location_even, mark_size, shift, hexes_width, hexes_hight, map_path):
    image = Image.open(map_path)
    image_copy = image.copy()
    for i in range(1, hexes_width, 2):
        for j in range(1, hexes_hight):
            canvas = Image.new('RGB', (mark_size, mark_size), 'cyan')
            image_copy.paste(canvas, (location_odd[0] + (i - 1) * shift[0],
                                      location_odd[1] + (j - 1) * shift[1]))
    for i in range(2, hexes_width, 2):
        for j in range(1, hexes_hight):
            canvas = Image.new('RGB', (mark_size, mark_size), 'cyan')
            image_copy.paste(canvas, (location_even[0] + (i - 1) * shift[0],
                                      location_even[1] + (j - 1) * shift[1]))
    image_copy.show()

In [186]:
# CONFIG ZONE
# YOUR LOCAL SYSTEM INFO
dir_path = 'replays_wl/'
files = ["347",
"349",
"399",
"405",
"549",
"597",
"619",
"620",
"627",
"647",
"658",
"670",
"680",
"707",
"772",
"773",
"776",
"777",
"780",
"785",
"841",
"849",
"851",
"865",
"868",
"870",
"871",
"874",
"890",
"930",
"1015",
"1031",
"1043",
"1046",
"1050",
"1052",
"1054",
"1063",
"1068",
"1083",
"1103",
"1114",
"1234",
"1243",
"1265",
"1280",
"1283",
"1325",
"1328",
"1500",
"1504"]
map_path = 'cave_rivers.png'
warm_map_template_name = 'caves_rivers'

# MAP INFO
hexes_width = 31
hexes_hight = 38

# NEED SOME TESTS IN NEXT CELLS
mark_size = 20
main_shift =  (54, 73)
pixel_shift_odd = (47, 50)
pixel_shift_even = (47, 75)


In [173]:
test_mark(
    mark_size=mark_size, # you can change
    location=(110, 150), # you can change
    map_path=map_path)

In [176]:
test_shift(
    shift = (54, 35), # you can change
    mark_size=mark_size, # your best from test_mark
    location=(110, 155), # your best from test_mark
    map_path=map_path
)

In [180]:
test_odd_hexes(
    location_odd=(47, 50), # you can change
    mark_size=mark_size, # your best from test_mark
    shift=(54, 72), # your best from test_shift
    hexes_width=hexes_width,
    hexes_hight=hexes_hight,
    map_path=map_path
)


In [183]:
test_even_hexes(
    location_even=(47, 75), # you can change
    mark_size=mark_size, # your best from test_mark
    shift=(54, 72), # your best from test_shift
    hexes_width=hexes_width,
    hexes_hight=hexes_hight,
    map_path=map_path
)

In [185]:
test_all(
    location_odd=(47, 50), # your best from test_odd_hexes
    location_even=(47, 75), # your best from test_even_hexes
    mark_size=mark_size, # your best from test_mark
    shift=(54, 73), # your best from test_shift
    hexes_width=hexes_width,
    hexes_hight=hexes_hight,
    map_path=map_path
)

In [187]:
generate_all_maps(dir_path=dir_path, files=files, map_path=map_path, new_map_name=warm_map_template_name,
                  show=False, hexes_width=hexes_width, hexes_hight=hexes_hight,
                  mark_size=mark_size, pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift, alpha=0.65
)

{'841', '1504', '1068', '773', '1054', '780', '658', '707', '1103', '1500', '670', '1052', '1063', '1234', '647', '772', '851', '627', '776', '865', '399', '890', '1328', '349', '1325', '1015', '1043', '871', '1114', '1280', '868', '1283', '849', '1243', '777', '785', '1050', '347', '1083', '1046', '874', '405', '930', '680', '870', '1265', '1031', '549', '619', '620', '597'}
{'841', '1504', '1068', '773', '1054', '780', '658', '707', '1103', '1500', '670', '1052', '1063', '1234', '647', '772', '851', '627', '776', '865', '399', '890', '1328', '349', '1325', '1015', '1043', '871', '1114', '1280', '868', '1283', '849', '1243', '777', '785', '1050', '347', '1083', '1046', '874', '405', '930', '680', '870', '1265', '1031', '549', '619', '620', '597'}
{'841', '1504', '1068', '773', '1054', '780', '658', '707', '1103', '1500', '670', '1063', '1234', '647', '772', '851', '627', '776', '865', '399', '890', '1328', '349', '1325', '1015', '1043', '871', '1114', '1280', '868', '1283', '849', '12

In [106]:
str(None)

'None'

In [139]:
# CONFIG ZONE
# YOUR LOCAL SYSTEM INFO
dir_path = 'replays_wl/'
files = ["343",
"354",
"356",
"358",
"363",
"410",
"416",
"423",
"560",
"573",
"599",
"612",
"629",
"650",
"651",
"657",
"685",
"698",
"788",
"789",
"792",
"860",
"861",
"866",
"867",
"872",
"884",
"888",
"889",
"899",
"900",
"906",
"924",
"929",
"1019",
"1025",
"1039",
"1082",
"1108",
"1266"]
map_path = 'autumn_harvest.png'
warm_map_template_name = 'ah'

# MAP INFO
hexes_width = 43
hexes_hight = 31

# NEED SOME TESTS IN NEXT CELLS
main_forward_shift = 27 # cool
main_down_shift = 36 # cool
main_shift =  (27, 36)
pixel_shift_odd = (10, 32)
pixel_shift_even = (10, 48)


generate_all_maps(dir_path=dir_path, files=files, map_path=map_path, new_map_name=warm_map_template_name,
                  show=False, hexes_width=hexes_width, hexes_hight=hexes_hight,
                  mark_size=15, pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift, alpha=0.65
)

{'770', '775', '869', '895', '783', '922', '1478', '925', '784', '782', '781', '913', '875'}
{'770', '775', '869', '895', '783', '922', '1478', '925', '784', '782', '781', '913', '875'}
{'770', '775', '869', '895', '783', '922', '1478', '925', '784', '782', '781', '913', '875'}
{'770', '775', '869', '895', '783', '922', '1478', '925', '784', '782', '781', '913', '875'}
{'770', '775', '869', '895', '783', '922', '1478', '925', '784', '782', '781', '913', '875'}
